In [79]:
#Importing Necessary Libraries
import pandas as pd
import geopandas as gpd
import folium 
from geopy.geocoders import Nominatim

In [80]:
#checking the availabe data sets in geo pandas
gpd.datasets.available

['naturalearth_cities', 'naturalearth_lowres', 'nybb']

In [81]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world.head()

,pop_est,continent,name,iso_a3,gdp_md_est,geometry
0,920938,Oceania,Fiji,FJI,8374.0,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000..."
1,53950935,Africa,Tanzania,TZA,150600.0,"POLYGON ((33.90371 -0.95000, 34.07262 -1.05982..."
2,603253,Africa,W. Sahara,ESH,906.5,"POLYGON ((-8.66559 27.65643, -8.66512 27.58948..."
3,35623680,North America,Canada,CAN,1674000.0,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742..."
4,326625791,North America,United States of America,USA,18560000.0,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000..."


In [82]:
#extracting continent europe from world data set
europe = world.loc[world.continent == 'Europe'].reset_index(drop = True)
europe.head()

,pop_est,continent,name,iso_a3,gdp_md_est,geometry
0,142257519,Europe,Russia,RUS,3745000.0,"MULTIPOLYGON (((178.72530 71.09880, 180.00000 ..."
1,5320045,Europe,Norway,-99,364700.0,"MULTIPOLYGON (((15.14282 79.67431, 15.52255 80..."
2,67106161,Europe,France,-99,2699000.0,"MULTIPOLYGON (((-51.65780 4.15623, -52.24934 3..."
3,9960487,Europe,Sweden,SWE,498100.0,"POLYGON ((11.02737 58.85615, 11.46827 59.43239..."
4,9549747,Europe,Belarus,BLR,165400.0,"POLYGON ((28.17671 56.16913, 29.22951 55.91834..."


In [83]:
# seperating stats and boundries
europe_stats = europe[['name','pop_est','gdp_md_est']]
europe_boundaries = europe[['name','geometry']]

In [84]:
europe_stats.head()

,name,pop_est,gdp_md_est
0,Russia,142257519,3745000.0
1,Norway,5320045,364700.0
2,France,67106161,2699000.0
3,Sweden,9960487,498100.0
4,Belarus,9549747,165400.0


In [85]:
europe_boundaries.head()

,name,geometry
0,Russia,"MULTIPOLYGON (((178.72530 71.09880, 180.00000 ..."
1,Norway,"MULTIPOLYGON (((15.14282 79.67431, 15.52255 80..."
2,France,"MULTIPOLYGON (((-51.65780 4.15623, -52.24934 3..."
3,Sweden,"POLYGON ((11.02737 58.85615, 11.46827 59.43239..."
4,Belarus,"POLYGON ((28.17671 56.16913, 29.22951 55.91834..."


In [86]:
#merging boundaries and stats
combined = europe_boundaries.merge(europe_stats, on ='name')

In [87]:
combined.head()

,name,geometry,pop_est,gdp_md_est
0,Russia,"MULTIPOLYGON (((178.72530 71.09880, 180.00000 ...",142257519,3745000.0
1,Norway,"MULTIPOLYGON (((15.14282 79.67431, 15.52255 80...",5320045,364700.0
2,France,"MULTIPOLYGON (((-51.65780 4.15623, -52.24934 3...",67106161,2699000.0
3,Sweden,"POLYGON ((11.02737 58.85615, 11.46827 59.43239...",9960487,498100.0
4,Belarus,"POLYGON ((28.17671 56.16913, 29.22951 55.91834...",9549747,165400.0


In [88]:
#creating a map
Map = folium.Map(location=[54,15],tiles = 'openstreetmap',zoom_start=3)
Map

In [102]:
#using geocoder to get latitude and longitude of the countries
geolocator = Nominatim(user_agent = "Suman")
#using lamda function to apply geocoder to countries in europe
def location(row):
    try:
        point = geolocator.geocode(row).point
        return pd.Series({'Latitude':point.latitude,'Longitude': point.longitude})
    except:
        return None

combined[['latitude','longitude']] = combined.apply(lambda x: location(x['name']),axis = 1)     

In [103]:
combined.head()

,name,geometry,pop_est,gdp_md_est,"(latitude, longitude)",latitude,longitude
0,Russia,"MULTIPOLYGON (((178.72530 71.09880, 180.00000 ...",142257519,3745000.0,None,64.686314,97.745306
1,Norway,"MULTIPOLYGON (((15.14282 79.67431, 15.52255 80...",5320045,364700.0,None,60.500021,9.099972
2,France,"MULTIPOLYGON (((-51.65780 4.15623, -52.24934 3...",67106161,2699000.0,None,46.603354,1.888334
3,Sweden,"POLYGON ((11.02737 58.85615, 11.46827 59.43239...",9960487,498100.0,None,59.674971,14.520858
4,Belarus,"POLYGON ((28.17671 56.16913, 29.22951 55.91834...",9549747,165400.0,None,53.425061,27.697136


In [105]:
for idx, row in combined.iterrows():
    folium.Marker([row['latitude'],row['longitude']],popup=row['name']).add_to(Map)

In [106]:
Map